In [75]:
from selenium import webdriver  # 동적크롤링
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [76]:
import time

import warnings
warnings.filterwarnings('ignore')

In [85]:
driver = webdriver.Chrome() 

In [86]:
url = "https://map.kakao.com/"
driver.get(url)

In [87]:
searchloc = '당진'

search_area = driver.find_element(By.XPATH, r'//*[@id="search.keyword.query"]') # 카카오맵 검색창
search_area.send_keys(searchloc)  # 검색어 전달
driver.find_element(By.XPATH, r'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # 돋보기 클릭

time.sleep(5)

driver.find_element(By.XPATH, r'//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER)  # 장소 탭

In [88]:
rv_list = []  # 정보 저장

In [89]:
def rvNamePrint():
    time.sleep(0.2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    rv_lists = soup.select('.placelist > .PlaceItem') # 장소 정보를 모두 가져옴
    for i, space in enumerate(rv_lists):
        temp = []
        
        name = space.select('.head_item > .tit_name > .link_name')[0].text
        score = space.select('.rating > .score > em')[0].text
        addr = space.select('.addr > p')[0].text
        
        # 상세정보 탭으로 이동
        driver.find_element(By.XPATH, r'//*[@id="info.search.place.list"]/li['+str(i+1)+']/div[5]/div[4]/a[1]').send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(2)
        


        rev = extract_review()  # 리뷰 추출 함수 실행
        
        
        # 하나의 리스트로 만들어 rv_list에 추가
        temp.append(name)
        temp.append(score)
        temp.append(addr[3:])
        temp.append(rev)
        
        rv_list.append(temp)

In [90]:
#new
def extract_review():
    # 후기 더보기 버튼을 클릭하는 루프
    while True:
        try:
            # 후기 더보기 버튼 클릭
            
            more_button = driver.find_element(By.LINK_TEXT, "후기 더보기")
            if '후기 접기' in more_button.text: # 마지막 후기 더보기 클릭 후 후기 접기를 교차로 반복 클릭하는 현상 발견하여 후기 접기 클릭 시 종료
                break
            
            #time.sleep(2)
            more_button.click()  # 클릭
            time.sleep(2)  # 페이지 로딩 대기
        except Exception as e:
            # 더보기 버튼이 없거나 클릭 실패 시 종료
            print("NEXT PLACE", e)
            break

    # 페이지 소스를 가져와서 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 후기 목록 찾기
    review_lists = soup.select('.list_evaluation > li')
    rev = []
    
    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for review in review_lists:
            comment = review.select('.txt_comment > span')[0].text  # 리뷰
            if len(comment) != 0:
                rev.append(comment)
    # 없으면 빈칸 추가
    else:
        rev.append(' ')
        
    # 다시 검색 탭으로 전환
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(2)
    
    return rev


In [91]:
page = 1  # 현재 페이지
page2 = 1  # 5개 중 몇번째인지(버튼이 5개씩있어서 6번째가 되면 다시 1로 바꿔줘야함)

# 한 페이지 당 15개의 장소
# 상위 45개 장소만 추출 

for i in range(1, 4):
    try:
        page2 += 1
        print(page, 'page')
        
        # 페이지 버튼 번호(1에서 5 사이 값)
        #if i > 5:
        #    xpath = '/html/body/div[5]/div[2]/div[1]/div[7]/div[6]/div/a['+str(i-5)+']'
        #else:
        xpath = '/html/body/div[5]/div[2]/div[1]/div[7]/div[6]/div/a['+str(i)+']'
        
        driver.find_element(By.XPATH, xpath).send_keys(Keys.ENTER)  # 페이지 선택
        
        rvNamePrint()  #  정보 크롤링
        
        
        # page2가 5를 넘어가면 다시 1로 바꿔주고 다음 버튼 클릭
        #if page2 > 5:
        #    page2 = 1
        #    driver.find_element(By.XPATH, r'//* [@id="info.search.page.next"]').send_keys(Keys.ENTER)
        
        page += 1
    
    except:
        break
        
print('크롤링 완료')

1 page
NEXT PLACE Message: no such element: Unable to locate element: {"method":"link text","selector":"후기 더보기"}
  (Session info: chrome=128.0.6613.138); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6DEF79412+29090]
	(No symbol) [0x00007FF6DEEEE239]
	(No symbol) [0x00007FF6DEDAB1DA]
	(No symbol) [0x00007FF6DEDFEFE7]
	(No symbol) [0x00007FF6DEDFF23C]
	(No symbol) [0x00007FF6DEE497C7]
	(No symbol) [0x00007FF6DEE2672F]
	(No symbol) [0x00007FF6DEE465A2]
	(No symbol) [0x00007FF6DEE26493]
	(No symbol) [0x00007FF6DEDF09D1]
	(No symbol) [0x00007FF6DEDF1B31]
	GetHandleVerifier [0x00007FF6DF29871D+3302573]
	GetHandleVerifier [0x00007FF6DF2E4243+3612627]
	GetHandleVerifier [0x00007FF6DF2DA417+3572135]
	GetHandleVerifier [0x00007FF6DF035EB6+801862]
	(No symbol) [0x00007FF6DEEF945F]
	(No symbol) [0x00007FF6DEEF4FB4]
	(No symbol) [0x00007FF6DEEF5140]
	(No symbo

In [72]:
rv_list

[['활옥동굴',
  '3.8',
  '충주시 목벌안길 26',
  ['한여름   그러나 그안은  한 가을  ㅎㅎ나가기 싫었어요  시원  재미  흥미  사진이 화려하게 나오네요^^',
   '여름에 피서하기 딱이에요!',
   '첨엔 재밌다가 가면갈수록 재미없음....겁나 춥고 냄시남특히 고추냉이 냄시',
   '동굴이 에어컨 가동중인것 만큼 시원하니 여름마다 방문하고 싶음',
   '활옥동굴 안 간이화장실 냄새  많이 나요, 반드시 입장 전에  화장실 다녀오시고, 더운 날씨에 가족들과 방문하기 괜찮아요.',
   '한 여름에도 긴팔 옷 필수 지참하세요',
   '한여름이라 상당히 시원해서 4점드립니다.볼거리는 딱히 없습니다.',
   '많이 선선합니다.겉옷 챙겨가시길근데 기대를 많이 했나뭔가 많이 아쉽네요. 유치한 조형물과 주제없이 그냥 이것저것 다 가져다 만들어놓은 느낌.과거 역사를 알수 있는 사진들은 한자 투성이에특색있는 공간을 잘 살리지 못해 많이 안타까웠네요. 이대로라면 재방문 의사 없네요.',
   '폭염속에 입구앞만 가도 시원한 공기가 쏟아져 나온다. 인공동굴이라 굴이 넓고 높다. 하지만 자연동굴 처럼 우와~ 하면서 걸어갈게 없다보니 어느시점부턴 지루하다...주말엔 교통정체 감안하고 가야함',
   '시원한디 내부는 구려요그리고 와사비 아이스크림 먹지마세요 바닐라 아이스크림 끝에 생와사비맛 나서 와사비 앵간히 좋아하는 사람 아니고선..',
   '무더운 여름 가면 좋은 곳이예요동굴안이 너무 시원해요주차장도 잘 구비되여서 먼주차장에 주차해도 셔틀버스운행하니 너무 편해요무더운 여름 추천합니다',
   '동굴 색감이 이뻤고 일단 시원했습니다.ㅡㅡㅋ 전 긴팔대신 쿨토시 장착하고 봤는데 열많은 타입은 반팔로도 조금 춥게 느껴질정도 (중간에 토시 뺐어요)',
   "최근에 가족과 함께 충주 활옥동굴을 방문했어요. 수도권에서 가까워서 드라이브하기 좋은 코스였고, 예전에는 활석과 옥석을 채광하던 광산이라 '활옥동굴'이라는 이름이 붙었다고 하네요. 동굴 입구부터 

In [92]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_records(rv_list)
df.to_excel(f'{searchloc}.xlsx')